## 获取分钟频交易量数据

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

def get_all_minute(type=None):
    '''获取某一年所有的分钟级别数据, 储存为一个三重multi-index df对象, 
    或者获取一个特定类型价格的二重multi-index df对象'''
    year = '2021'
    folder_root = '/mnt/datadisk/pjluo_shared/minute_h5_data/'
    folder = Path(folder_root)
    f_names = [f.name for f in folder.iterdir() if f.is_file() and f.name.endswith('.h5')]
    f_names = [f for f in f_names if f.startswith(year)]
    f_names.sort(key=lambda x: int(x[:8]))
    
    df_dic = {}
    for f in tqdm(f_names):
        df = pd.read_hdf(folder_root + f)
        df_reset = df.reset_index()
        df_multi = df_reset.set_index(['InstrumentID', 'EndTime'])
        if type == None:
            df_dic[f[:8]] = df_multi
        else:
            df_dic[f[:8]] = df_multi[type].unstack().T

    all_minute = pd.concat(df_dic.values(), keys=df_dic.keys())

    if type == None:
        all_minute.index.names = ['Date','InstrumentID', 'EndTime']
    else:
        all_minute.index.names = ['Date', 'EndTime']
        
    return all_minute

In [3]:
vol = get_all_minute('LastVolume')

In [4]:
vol

SH600000  SH600004    SH600006  SH600007   SH600008  \
Date     EndTime                                                         
20210104 09:31    1727980.0  257345.0  10138000.0   30600.0  1489000.0   
         09:32     865050.0  255293.0   5092700.0    3400.0   718700.0   
         09:33     877830.0  172506.0   6881859.0    4800.0   317300.0   
         09:34     835670.0  225664.0   3533000.0   11100.0   165400.0   
         09:35     436000.0  164867.0   2236900.0   23900.0   598760.0   
...                     ...       ...         ...       ...        ...   
20211231 14:56     128400.0   78100.0     85900.0   25300.0   414900.0   
         14:57     168900.0  110100.0    149000.0    7250.0   496500.0   
         14:58          0.0       0.0         0.0       0.0        0.0   
         14:59          0.0       0.0         0.0       0.0        0.0   
         15:00     495800.0  108900.0    227500.0   47500.0  1065500.0   

                  SH600009    SH600010   SH600011  SH600012   SH600015  ...  \
Date     EndTime                                                        ...   
20210104 09:31    390317.0   3122500.0  1089300.0  145500.0  1319700.0  ...   
         09:32    325080.0   2368340.0   638800.0   47500.0   443500.0  ...   
         09:33    256800.0   1626100.0   760900.0   21300.0   214400.0  ...   
         09:34    159200.0   1754700.0   571700.0   33000.0   169100.0  ...   
         09:35    347600.0   1981840.0  1363400.0   45400.0   615100.0  ...   
...                    ...         ...        ...       ...        ...  ...   
20211231 14:56     75700.0  26454705.0   693800.0   32400.0    93000.0  ...   
         14:57     54550.0   2303100.0   438700.0   30400.0    65229.0  ...   
         14:58         0.0         0.0        0.0       0.0        0.0  ...   
         14:59         0.0         0.0        0.0       0.0        0.0  ...   
         15:00     98654.0   5903500.0  2321599.0   85200.0   509800.0  ...   

                  SZ301190  SH688210  SH688206  SH688265  SZ301166  SZ301189  \
Date     EndTime                                                               
20210104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56    163600.0   11984.0   59978.0   15267.0   34897.0   50300.0   
         14:57    137200.0   14541.0  227516.0   25547.0   87700.0  149100.0   
         14:58       500.0       0.0       0.0       0.0    3000.0    3300.0   
         14:59         0.0       0.0       0.0       0.0       0.0       0.0   
         15:00    241900.0   23659.0  242271.0   20106.0  134900.0  143951.0   

                  SH688227  SH688236  SZ301127  SH603176  
Date     EndTime                                          
20210104 09:31         NaN       NaN       NaN       NaN  
         09:32         NaN       NaN       NaN       NaN  
         09:33         NaN       NaN       NaN       NaN  
         09:34         NaN       NaN       NaN       NaN  
         09:35         NaN       NaN       NaN       NaN  
...                    ...       ...       ...       ...  
20211231 14:56     49753.0   32963.0  287399.0    4300.0  
         14:57     53824.0   76664.0  619224.0       0.0  
         14:58         0.0       0.0    3100.0       0.0  
         14:59         0.0       0.0       0.0       0.0  
         15:00     75860.0  110136.0  553124.0    8532.0  

[58320 rows x 4622 columns]

In [5]:
vol.to_hdf('vol_2021.h5', key='df')